# Lession 10 假新聞辨識

In [ ]:
import os
os.chdir('/tmp/Lession10')

## 資料前處理

In [ ]:
%pip install pandas

### 讀取檔案

In [ ]:
import pandas as pd

In [ ]:
true = pd.read_csv('Data/True.csv')
fake = pd.read_csv('Data/Fake.csv')

In [ ]:
true

In [ ]:
fake

### 設定Label

In [ ]:
true['label'] = 1
fake['label'] = 0

### 合併&丟棄不會用到的欄位

In [ ]:
data = pd.concat([true,fake], axis=0).drop(columns=['date','subject']).dropna().sample(frac=1).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
train = data[:30000]
test = data[30000:]

In [ ]:
trainSentences = train['title'].tolist()
testSentences = test['title'].tolist()

## Tokenize Dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 5000
embedding_dim = 16
max_length = 500
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(trainSentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(trainSentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type)

testing_sequences = tokenizer.texts_to_sequences(testSentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

## Build Model

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(300, dropout=0.3, recurrent_dropout=0.3)
))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
history = model.fit(training_padded, train['label'], epochs=50, validation_data=(testing_padded, test['label']), callbacks=[earlystop])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(data_model.history['loss'])
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
plt.plot(data_model.history['val_loss'])
plt.title('Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Val Loss')

In [ ]:
plt.plot(data_model.history['accuracy'])
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acuracy')

In [ ]:
plt.plot(data_model.history['val_accuracy'])
plt.title('Val Acurracy')
plt.xlabel('Epochs')
plt.ylabel('Val Accuracy')